In [19]:
import math
import metapy
import pytoml
import csv
import pandas as pd
import os
import shutil


In [2]:
df = pd.read_csv ('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_lessons.csv')
print(df.head())

  course_id  week_nbr  video_id  \
0    cs-410         1         1   
1    cs-410         1         2   
2    cs-410         1         3   
3    cs-410         1         4   
4    cs-410         1         5   

                                             content  
0  Hello welcome to CS410 DSO Text Information Sy...  
1  [SOUND] >> This lecture is about Natural Langu...  
2  [SOUND] In this lecture,we're going to talk ab...  
3  [MUSIC] This lecture is aboutthe text retrieva...  
4  [SOUND] This lecture is a overview oftext retr...  


In [3]:
print(df.columns)

Index(['course_id', 'week_nbr', 'video_id', 'content'], dtype='object')


In [4]:
segcolumns = ['content']

In [5]:
new_df_text = pd.DataFrame(df, columns=segcolumns)

In [6]:
print(new_df_text)

                                               content
0    Hello welcome to CS410 DSO Text Information Sy...
1    [SOUND] >> This lecture is about Natural Langu...
2    [SOUND] In this lecture,we're going to talk ab...
3    [MUSIC] This lecture is aboutthe text retrieva...
4    [SOUND] This lecture is a overview oftext retr...
..                                                 ...
275  So, now that we understand that logistic regre...
276  Now, we'll take a longer look at using logisti...
277  Is a credit card transactionreal or fraudulent...
278  So, now we've seen how logistic regression cou...
279  So now we'll go ahead and take a look at a ver...

[280 rows x 1 columns]


In [7]:
print(new_df_text)

                                               content
0    Hello welcome to CS410 DSO Text Information Sy...
1    [SOUND] >> This lecture is about Natural Langu...
2    [SOUND] In this lecture,we're going to talk ab...
3    [MUSIC] This lecture is aboutthe text retrieva...
4    [SOUND] This lecture is a overview oftext retr...
..                                                 ...
275  So, now that we understand that logistic regre...
276  Now, we'll take a longer look at using logisti...
277  Is a credit card transactionreal or fraudulent...
278  So, now we've seen how logistic regression cou...
279  So now we'll go ahead and take a look at a ver...

[280 rows x 1 columns]


In [8]:
corpus = []
for index, row in df.iterrows():
    text = row['content']
    corpus.append(text)

In [9]:
with open('course.dat', 'w') as f:
    for text in corpus:
        f.write(text)
        f.write('\n')

In [12]:
idx = metapy.index.make_inverted_index('config.toml')
print(idx)

In [17]:
print ("directory exists:" + str(os.path.isdir('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/idx')))


directory exists:True


In [24]:
if os.path.isdir('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/idx'):
    path = '/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/idx'
    shutil.rmtree(path)
    
src = '/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/course.dat'
dst = '/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/course'

shutil.copy2(src,dst)

'/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/course/course.dat'

In [25]:
idx = metapy.index.make_inverted_index('config.toml')

In [26]:
idx.num_docs()

280

In [27]:
idx.unique_terms()

15626

In [28]:
idx.avg_doc_length()

1141.0142822265625

In [29]:
idx.total_corpus_terms()

319484

In [30]:
ranker = metapy.index.OkapiBM25()

In [31]:
query = metapy.index.Document()
query.content('PLSA')

In [32]:
top_docs = ranker.score(idx, query, num_results=5)
top_docs

[(68, 8.984014511108398), (67, 7.510433197021484), (73, 6.776474475860596)]

In [33]:
for num, (d_id, _) in enumerate(top_docs):
    print(idx.metadata(d_id).get(;))


SyntaxError: invalid syntax (<ipython-input-33-42d675f73bab>, line 2)

In [34]:
stri = '/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_lessons.csv'
print(stri)

/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_lessons.csv


In [35]:
print(stri.rfind('/'))

61


In [36]:
print(stri[0: stri.rfind('/')+1])

/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/
